In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
df = pd.read_csv("Churn_Modelling.csv")

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#preprocessing 
df = df.drop(["RowNumber","CustomerId","Surname"],axis = 1)

In [5]:
df["Geography"].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [6]:
df.shape

(10000, 11)

In [7]:
df.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [8]:
## label encoding
label_encoder = LabelEncoder()
df["Gender"] = label_encoder.fit_transform(df["Gender"])

In [9]:
## one hot encoding for geography columns
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output=False)
## sparse should be False
geo_enc = one_hot_encoder.fit_transform(pd.DataFrame(df["Geography"]))

In [10]:
geo_enc.shape

(10000, 3)

In [11]:
cols = one_hot_encoder.get_feature_names_out(["Geography"])

In [12]:
geo_df = pd.DataFrame(geo_enc, columns=cols)

In [13]:
df = df.drop(["Geography"],axis=1)

In [14]:
df = pd.concat([df,geo_df],axis = 1)

In [15]:
df.isna().sum()

CreditScore          0
Gender               0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Geography_France     0
Geography_Germany    0
Geography_Spain      0
dtype: int64

In [16]:
## save the encoders and sscaler
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder,file)
with open("one_hot_encoder_geo.pkl","wb") as file:
    pickle.dump(one_hot_encoder,file)

In [17]:
## devide the data set into dependent and independent features
X = df.drop("Exited",axis=1)
y = df["Exited"]
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [18]:
## splitting train and test data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.75,random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test= scaler.fit_transform(X_test)

In [19]:
X_train,X_test

(array([[ 0.46994759, -1.12085578,  0.8687205 , ...,  0.99123838,
         -0.56503205, -0.58289296],
        [ 1.62363075, -1.12085578, -1.02424669, ..., -1.00883906,
         -0.56503205,  1.71558085],
        [-0.44251091,  0.89217544,  0.11153363, ...,  0.99123838,
         -0.56503205, -0.58289296],
        ...,
        [ 0.86849268, -1.12085578, -0.07776309, ...,  0.99123838,
         -0.56503205, -0.58289296],
        [ 0.15530673,  0.89217544,  0.39547871, ...,  0.99123838,
         -0.56503205, -0.58289296],
        [ 0.46994759,  0.89217544,  1.15266558, ..., -1.00883906,
          1.76981112, -0.58289296]]),
 array([[-0.55602358,  0.91924885, -0.66487639, ..., -1.00080032,
          1.71437325, -0.57078109],
        [-0.27787993,  0.91924885,  0.38665919, ...,  0.99920032,
         -0.58330355, -0.57078109],
        [-0.50451549, -1.08784471,  0.48225333, ..., -1.00080032,
         -0.58330355,  1.75198515],
        ...,
        [ 1.2879658 , -1.08784471,  2.58532449, ...,  

In [20]:
# saving the scaler file in pickle form
with open("sscaler.pkl",'wb') as file:
    pickle.dump(scaler,file)

In [27]:
X_train.shape

(2500, 12)

In [26]:
# ANN implementation
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [29]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

c:\Users\LENOVA\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.02345)

In [40]:
model.compile(optimizer = opt,loss="binary_crossentropy",metrics =['accuracy'])

In [41]:
## set up the tensorboard
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [43]:
tensorflow_callback = TensorBoard(log_dir = log_dir,histogram_freq = 1)

In [48]:
early_stopping_callback =  EarlyStopping(monitor = 'val_loss',
                                         patience = 15, 
                                         restore_best_weights=True)

In [49]:
## training the model
history = model.fit(X_train,y_train,
                    validation_data = (X_test,y_test),
                    epochs= 100,
                    callbacks = [tensorflow_callback,early_stopping_callback]
                    )

Epoch 1/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8449 - loss: 0.3747 - val_accuracy: 0.8473 - val_loss: 0.3990
Epoch 2/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8501 - loss: 0.3724 - val_accuracy: 0.8548 - val_loss: 0.3619
Epoch 3/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8610 - loss: 0.3352 - val_accuracy: 0.8441 - val_loss: 0.3718
Epoch 4/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8469 - loss: 0.3493 - val_accuracy: 0.8497 - val_loss: 0.3705
Epoch 5/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8598 - loss: 0.3250 - val_accuracy: 0.8524 - val_loss: 0.3812
Epoch 6/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8582 - loss: 0.3328 - val_accuracy: 0.8463 - val_loss: 0.3729
Epoch 7/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8490 - loss: 0.3463 - val_accuracy: 0.8575 - val_loss: 0.3590
Epoch 8/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8569 - loss: 0.3303 - val_accuracy: 0.8503 

In [50]:
model.save("model.h5")

In [55]:
## load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [56]:
%tensorboard --logdir logs/fit20241217-151042

Reusing TensorBoard on port 6008 (pid 7304), started 0:01:47 ago. (Use '!kill 7304' to kill it.)